## Essai sur la tarification non vie et approche causale

Exploration de la base de données

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [36]:
DATA_PATH = r"C:\Users\KDO\Downloads\Data TP Essfar Tarification automobile"
FILE_Contrat= os.path.join(DATA_PATH, "base_contrat_tpessfar.csv")
FILE_sinistre= os.path.join(DATA_PATH, "base_sinistre_tpessfar.csv")
contrat = pd.read_csv(FILE_Contrat, sep=';', index_col=0, na_values=['NA'], encoding='latin1', nrows=50)
sinistre=pd.read_csv(FILE_sinistre, sep=';', index_col=0, na_values=['NA'], encoding='latin1', nrows=50)


In [39]:
contrat.head(10)

,NSOC,NRIS,CPER,DEBUT,FIN,DNAICDD,DPERCDD,ANCVEH,CFORASU,CGEOMAIF,...,QPORVEH,MVALVEH,DFINFABMOD,CSEXE,CPOS,CEMP,CDOMACT,CPROSOC,SFAM,QENF
5551055,2399420,6,1,01JAN2015,31DEC2015,05JUN1994,30SEP2013,2,3,60159,...,3,13850.0,18AUG2014,2,60350,1,17,8,M,1
10482056,6161071,3,1,01JAN2014,31DEC2014,12DEC1980,08JUL1999,9,5,21166,...,5,24800.0,01JUL2005,1,21220,4,37,7,P,3
1103197,581292,8,1,01JAN2014,31DEC2014,04MAR1936,13MAY1958,2,1,30047,...,5,17850.0,05NOV2012,2,30230,1,1,8,M,0
8142121,3663307,1,1,01JAN2015,31DEC2015,29MAY1987,31AUG2014,14,5,93079,...,5,16145.0,01DEC2000,2,93430,2,7,13,U,0
11611723,6846015,5,1,29NOV2014,31DEC2014,04APR1988,04APR2008,9,2,94077,...,5,19600.0,01JAN2005,1,94290,4,42,13,M,2
7496503,3370763,1,1,01APR2015,31DEC2015,03AUG1978,13DEC1996,14,5,14208,...,3,18294.0,01DEC2004,1,14370,1,3,8,C,0
4073096,1715183,2,1,01JAN2014,02JUL2014,05JUN1950,23SEP1970,12,3,35055,...,5,10300.0,01MAY2003,2,35135,1,0,13,C,0
3496809,1446087,1,1,28OCT2014,31DEC2014,20MAY1938,22MAR1962,0,3,94080,...,5,27500.0,02FEB2015,2,94300,3,13,9,M,0
7283172,3235799,3,1,01JAN2014,31DEC2014,28MAY1954,16OCT1974,12,5,35220,...,5,17150.0,01AUG2002,1,35150,1,17,7,M,0
11151910,6609417,3,1,21JUN2016,31DEC2016,11MAR1943,01MAR1965,0,1,5006,...,5,19650.0,NaN,2,5120,4,40,5,C,0


In [ ]:
sinistre.head(10)

In [43]:
def identifier_type_variable(series, seuil_discret=20):
    """
    Identifie le type d'une variable :
    - 'Quantitative continue'
    - 'Quantitative discrète'
    - 'Catégorielle'
    
    seuil_discret : nombre de valeurs uniques maximum pour considérer une variable entière comme discrète
    """
    if pd.api.types.is_numeric_dtype(series):
        if series.nunique() <= seuil_discret:
            return 'Quantitative discrète'
        else:
            return 'Quantitative continue'
    else:
        return 'Catégorielle'

# Appliquer la fonction à toutes les colonnes
types_variables = contrat.apply(identifier_type_variable)

# Afficher le résultat
print(types_variables)


NSOC          Quantitative continue
NRIS          Quantitative discrète
CPER          Quantitative discrète
DEBUT                  Catégorielle
FIN                    Catégorielle
DNAICDD                Catégorielle
DPERCDD                Catégorielle
ANCVEH        Quantitative discrète
CFORASU       Quantitative discrète
CGEOMAIF      Quantitative continue
CGRO                   Catégorielle
CNIVASP                Catégorielle
CNIVFRN       Quantitative discrète
CNIVOTR                Catégorielle
CNIVSVR                Catégorielle
CPEVANN       Quantitative discrète
CSINANN       Quantitative discrète
CTBR          Quantitative discrète
CUSARIS       Quantitative discrète
KCLAANN       Quantitative discrète
QECH050       Quantitative discrète
NREPVEH                Catégorielle
TCDD          Quantitative discrète
CCATVEH       Quantitative discrète
CENEVEH       Quantitative discrète
CGENVEH       Quantitative discrète
CSEGVEH                Catégorielle
CSRA          Quantitative d

In [44]:

from datetime import datetime

def audit_qualite(df, colonnes_doublons=['NRIS','NREPVEH'], date_col='DNAICDD', veh_col='ANCVEH'):
    """
    Audit qualité des données :
    - Valeurs manquantes (%)
    - Valeurs aberrantes (%)
        - Date de naissance futuriste ou âge <16 ans
        - Ancienneté véhicule négative
    - Doublons (%)
    
    Paramètres :
    df : DataFrame pandas
    colonnes_doublons : colonnes clés pour vérifier les doublons
    date_col : colonne date de naissance du conducteur
    veh_col : colonne ancienneté du véhicule
    
    Retourne : dictionnaire avec trois DataFrame : missing, aberrant, duplicates
    """
    n = len(df)
    
    # 1️⃣ Valeurs manquantes en %
    missing_percent = df.isnull().sum() / n * 100
    missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
    
    # 2️⃣ Valeurs aberrantes
    today_year = datetime.today().year
    aberrant_dict = {}
    
    # Age conducteur <16 ans
    try:
        age_conducteur = today_year - pd.to_datetime(df[date_col], errors='coerce').dt.year
        aberrant_dict[f'Conducteur <16 ans ({date_col})'] = (age_conducteur < 16).sum() / n * 100
    except:
        aberrant_dict[f'Conducteur <16 ans ({date_col})'] = 'Colonne inexistante'
    
    # Ancienneté véhicule négative
    if veh_col in df.columns:
        aberrant_dict[f'Ancienneté véhicule négative ({veh_col})'] = (df[veh_col] < 0).sum() / n * 100
    else:
        aberrant_dict[f'Ancienneté véhicule négative ({veh_col})'] = 'Colonne inexistante'
    
    aberrant_percent = pd.Series(aberrant_dict).sort_values(ascending=False)
    
    # 3️⃣ Doublons
    duplicates_percent = df.duplicated(subset=colonnes_doublons).sum() / n * 100
    
    duplicates_result = pd.DataFrame({
        'Colonnes': [", ".join(colonnes_doublons)],
        'Pourcentage doublons': [duplicates_percent]
    })
    
    return {
        'Missing Values (%)': missing_percent,
        'Aberrant Values (%)': aberrant_percent,
        'Duplicates (%)': duplicates_result
    }

# 🔹 Exemple d'utilisation

resultat = audit_qualite(contrat)
print(resultat['Missing Values (%)'])
print(resultat['Aberrant Values (%)'])
print(resultat['Duplicates (%)'])


QECH050       38.0
CCLAREP       38.0
CSRA          16.0
DFINFABMOD     8.0
SFAM           2.0
dtype: float64
Conducteur <16 ans (DNAICDD)             0.0
Ancienneté véhicule négative (ANCVEH)    0.0
dtype: float64
        Colonnes  Pourcentage doublons
0  NRIS, NREPVEH                   0.0
